This example was based on the lime example https://marcotcr.github.io/lime/tutorials/Lime%20-%20basic%20usage%2C%20two%20class%20case.html

In [17]:
import os, json, sys
sys.path.append('../..')

import sklearn
import numpy as np
import sklearn
import sklearn.ensemble
import sklearn.metrics

# Fetching data, training a classifier
For this tutorial, we'll be using the 20 newsgroups dataset. In particular, for simplicity, we'll use a 2-class subset: atheism and christianity.

In [11]:
from sklearn.datasets import fetch_20newsgroups

In [13]:
categories = ['alt.atheism', 'soc.religion.christian']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)
class_names = ['atheism', 'christian']
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=False)
train_vectors = vectorizer.fit_transform(newsgroups_train.data)
test_vectors = vectorizer.transform(newsgroups_test.data)

In [20]:
model = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
model.fit(train_vectors, newsgroups_train.target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [22]:
pred = model.predict(test_vectors)
print('F1 score:', sklearn.metrics.f1_score(newsgroups_test.target, pred, average='binary'))

F1 score: 0.9209302325581395


# Explanation

In [18]:
from m_lime.explanations.linear import ExplainLinear
from m_lime.explanations.visualization import ImagePlot
from m_lime.densities.density_vae import DensityVAE
from m_lime.densities.density_kde import DensityKDE
from m_lime.densities.density_kde_pca import DensityKDEKPCA, DensityKDEPCA

## Testing model

# Explaining a Document

In [89]:
idx = 83
# newsgroups_test.data[idx]
print('Document id: %d' % idx)
# print(model.predict(test_vectors))
print('Probability(christian) =', model.predict_proba(test_vectors[idx]))

print('True class: %s' % class_names[newsgroups_test.target[idx]])
print('True class number:',newsgroups_test.target[idx] )
print()
print('Text:')
print(newsgroups_test.data[idx])
x_explain = test_vectors[idx].reshape(1, -1).toarray().astype(float32)
document_explain = newsgroups_test.data[idx]

Document id: 83
Probability(christian) = [[0.534 0.466]]
True class: atheism
True class number: 0

Text:
From: johnchad@triton.unm.edu (jchadwic)
Subject: Another request for Darwin Fish
Organization: University of New Mexico, Albuquerque
Lines: 11
NNTP-Posting-Host: triton.unm.edu

Hello Gang,

There have been some notes recently asking where to obtain the DARWIN fish.
This is the same question I have and I have not seen an answer on the
net. If anyone has a contact please post on the net or email me.

Thanks,

john chadwick
johnchad@triton.unm.edu
or



NameError: name 'float32' is not defined

In [90]:
x_explain

array([[0., 0., 0., ..., 0., 0., 0.]])

# Generating an Explanation

In [33]:
import torch
from scipy.sparse import coo_matrix

In [74]:
train_vectors_ = torch.from_numpy(train_vectors.toarray().astype(np.float32), )
labels = torch.from_numpy(newsgroups_train.target)

In [75]:
train_vectors_.shape

torch.Size([1079, 23035])

In [76]:
train_vectors_

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [77]:
newsgroups_train.target

array([1, 0, 1, ..., 0, 1, 1])

In [78]:
print(train_vectors_.size()) #, newsgroups_train.target.size())

torch.Size([1079, 23035])


In [79]:
train = torch.utils.data.TensorDataset(train_vectors_, labels)
train_loader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=False)

In [80]:
input_dim = test_vectors[idx].shape[1]

In [81]:
density = DensityVAE(input_dim=input_dim)

In [82]:
density.fit(train_loader, epochs=20)

/opt/conda/conda-bld/pytorch_1582013226511/work/torch/csrc/utils/python_arg_parser.cpp:698: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, Number alpha)


In [91]:
torch.cuda.empty_cache()
class_to_explain = 0
print('Explanation for ', class_to_explain)
try:
    del explain_linear
except:
    pass
explain_linear = ExplainLinear(model_predict=model, density=density, linear_model='SGD', verbose=True)
result = explain_linear.explain_instance(
    x_explain=x_explain, r=0.1, class_index=class_to_explain, tol=0.1)
y = 'index: {:} - prediction: {:}'.format(index, np.argmax(y_explain))
fig, axis = plt.subplots(1, figsize=(4.5, 4.5))
ax = plot_instances(axis, x_explain, y)
a = ImagePlot.plot_importance(result['importance'], standardization=True)


Explanation for  0


RuntimeError: Expected object of scalar type Double but got scalar type Float for argument #2 'mat2' in call to _th_mm

In [ ]:
dict_exp = exp_NLS.get_text_explanation(x_explain, document=document_explain, num_features=10)

In [ ]:
def plot_betas(dict_exp):
    fig, ax = plt.subplots()
    width = 0.35 
    ind = np.arange(len(dict_exp['words'][::-1]))
    bar_pos = plt.barh(ind-width/2, dict_exp['betas_document'][::-1], width, label='Atheism')
    plt.title('Atheism Class Feature Importance')
    bar_neg = plt.barh(ind + width/2, dict_exp['betas_document_neg'][::-1], width, label='Christian')
    plt.title('Atheism Christian Class Feature Importance')
    ax.set_yticks(ind)
    ax.set_yticklabels(dict_exp['words'][::-1])
    ax.grid(True, axis='y')
    leg = plt.legend()

In [ ]:
plot_betas(dict_exp)

In [ ]:
plot_exp = exp_NLS.explain_graphical(x_explain, document=document_explain, num_features=8)


In [ ]:
names_features[dict_exp['indices_words']], dict_exp['indices_words']


In [ ]:
document_html = exp_NLS.document_html(x_explain, document=document_explain, num_features=20)

In [ ]:
document = dict_exp['document']

In [ ]:
import re
prohibitedWords = ['on', 'Random', 'Words']
big_regex = re.compile('|'.join(map(re.escape, prohibitedWords)))
# the_message = big_regex.sub("<replaced>", 'this message contains Some really Random Words')
text = re.sub(r"\b(%s)\b" % "|".join(words), replace, text)
the_message

In [ ]:
def replace(matched):
    # Matched.group(0) is the word that was found
    # Return the replacement
    return "REPLACEMENT"

In [ ]:
words = ['a', 'an']
text = 'A word here, an b a'
text = re.sub(r"\b(%s)\b" % "|".join(words), replace, text)
text

In [ ]:
def document_html(x_explain, document, num_features=10, tokenizer=None):
    exp = exp_NLS.get_text_explanation(x_explain, document, num_features=num_features)
    if tokenizer is None:
        return None
    document_html = document
    document_tokens = tokenizer(document)
    
    format_i = '<b style="background-color:Tomato;">' 
    format_e = '</b>'
    for words in exp['words']:
        if words in document_tokens:
            document_html = re.sub(r"\b(%s)\b" % "|".join(words), '{:}{:}{:}'.format(
                format_i, words, format_e), document_html)
#             document_html = re.sub(words , '{:}{:}{:}'.format(format_i, words, format_e) , document_html)
#     for words in document_tokens:
#         if words in exp['words']:
#             document_html += '<b>{:}</b>'.format(words)
#         else:
#             document_html += words
#         print(words)
        
    print('************************')
    print(document_html)
    return TextHTML(document_html)

In [ ]:
class TextHTML:
    def __init__(self, html):
        self.html = html
    def _repr_html_(self):
        return self.html

In [ ]:
def _segment_with_tokens(text, tokens):
    """Segment a string around the tokens created by a passed-in tokenizer"""
    list_form = []
    text_ptr = 0
    for token in tokens:
        inter_token_string = []
        while not text[text_ptr:].startswith(token):
            inter_token_string.append(text[text_ptr])
            text_ptr += 1
            if text_ptr >= len(text):
                raise ValueError("Tokenization produced tokens that do not belong in string!")
        text_ptr += len(token)
        if inter_token_string:
            list_form.append(''.join(inter_token_string))
        list_form.append(token)
    if text_ptr < len(text):
        list_form.append(text[text_ptr:])
    return list_form

In [ ]:
a = document_html(x_explain, document, num_features=10, tokenizer=tokenizer)

In [ ]:
a

# Explaining Lime

In [ ]:
sparse_matrix = SparseMatrix()

In [ ]:
c_1 = make_pipeline(vectorizer, sparse_matrix, model)

In [ ]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names, random_state=65464)

In [ ]:
exp = explainer.explain_instance(newsgroups_test.data[idx], c_1.predict_proba, num_features=10)

In [ ]:
exp.as_list()

In [ ]:
exp.show_in_notebook(text=True)

In [ ]:
plot_exp = exp_NLS.explain_graphical(x_explain, document=document_explain, num_features=10)

# Explanation from the Data

In [ ]:
# y_train_atheism = newsgroups_train.target[indices_atheism]
x_train = np.array(train_vectors)

In [ ]:
indices_atheism = np.where(newsgroups_train.target == 0)[0]
x_train_atheism = x_train[indices_atheism]
important_words = x_train_atheism[:, dict_exp['indices_all_words']]
sum_atheism = np.sum(important_words, axis=0)

In [ ]:
indices_christian = np.where(newsgroups_train.target == 1)[0]
x_train_christian = x_train[indices_christian]
important_words_christian = x_train_christian[:, dict_exp['indices_all_words']]
sum_christian = np.sum(important_words_christian, axis=0)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10), dpi=200)
ax.barh(names_features[dict_exp['indices_all_words']], sum_christian, label='christian')
ax.barh(names_features[dict_exp['indices_all_words']], sum_atheism, label='atheism', alpha=0.5)
leg = plt.legend()


In [ ]:
fig, ax = plt.subplots(figsize=(10, 15))
y_p = np.arange(len(names_features[dict_exp['indices_all_words']]))
vals = sum_atheism-sum_christian
colors = ['green' if x > 0 else 'red' for x in vals]
ax.barh(y_p, vals, color=colors)
ax.set_yticks(y_p)
ax.set_yticklabels(names_features[dict_exp['indices_all_words']])
x_lim = ax.get_xlim()
colors = ['tab:orange', 'tab:blue']
for i, y_p_i in enumerate(y_p):
    i_c = i%2
    ax.plot(x_lim, [y_p_i]*2, c=colors[i_c])
ax.set_xlim(x_lim)
title = ax.set_title('Atheism - Christain (frequency words)')
# leg = plt.legend()

# Removing: headers, footers, quotes

## Preparing data

In [ ]:
categories = ['alt.atheism', 'soc.religion.christian']
newsgroups_train_clean = fetch_20newsgroups(subset='train'
                                      , remove=('headers', 'footers', 'quotes')
                                      , categories=categories)
newsgroups_test_clean = fetch_20newsgroups(subset='test'
                                     , remove=('headers', 'footers', 'quotes')
                                     , categories=categories)
class_names = ['atheism', 'christian']
vectorizer_clean = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=True)  # ngram_range=(1, 2)
train_vectors_clean = vectorizer_clean.fit_transform(newsgroups_train_clean.data)
test_vectors_clean = vectorizer_clean.transform(newsgroups_test_clean.data)
train_vectors_clean = train_vectors_clean.toarray()
test_vectors_clean = test_vectors_clean.toarray()
tokenizer_clean = vectorizer_clean.build_tokenizer()

In [ ]:
names_features_clean = np.array(vectorizer_clean.get_feature_names())

In [ ]:
len(vectorizer_clean.get_feature_names())

## Training the NLS

In [ ]:
comb_parameters = [{
        'es_give_up_after_nepochs': 20
        , 'hidden_size': 100
        , 'num_layers': 2
        , 'n_classification_labels': 2,
    }
        ]

In [ ]:
for parameter in comb_parameters:
    model_clean = NLS(
        verbose=0
        , es=True
        , gpu=True
        , scale_data=False
        , varying_theta0=False
        , fixed_theta0=True
        , dataloader_workers=0
        # , with_mean=False
        , **parameter
    ) 
    model_clean.fit(x_train=train_vectors_clean, y_train=newsgroups_train_clean.target)

## Testing model

In [ ]:
pred = model_clean.predict(test_vectors_clean)
pred_poba = model_clean.predict_proba(test_vectors_clean)
print('Score:', sklearn.metrics.f1_score(newsgroups_test_clean.target, pred, average='binary'))

# Generating an Explanation

In [ ]:
# alterando

In [ ]:
exp_NLS_clean = ExplainText(model_clean
                             , class_names=['atheism', 'christian']
                             , names_features=names_features_clean
                            )

In [ ]:
dict_exp_clean = exp_NLS_clean.get_text_explanation(x_explain_clean
                                              , document=document_explain_clean
                                              , num_features=20)

In [ ]:
betas = model_clean.get_thetas(x_pred=x_explain_clean, net_scale=True)[2][0]

In [ ]:
x_explain_clean

In [ ]:
words_from_text_indices = np.argwhere(x_explain_clean[0] > 0).reshape(-1)

In [ ]:
names_doc = names_features_clean[words_from_text_indices]

In [ ]:
betas_r = betas[words_from_text_indices]

In [ ]:
betas_sum = np.abs(betas_r[:, 0] - betas_r[:, 1])

In [ ]:
indices = np.argsort(betas_sum)[::-1]
# names_doc[indices]

In [ ]:
plot_betas(dict_exp_clean)

In [ ]:
plot_exp = exp_NLS_clean.explain_graphical(x_explain_clean
                                           , document=document_explain_clean
                                           , num_features=20)


In [ ]:
names_features_clean[dict_exp_clean['indices_words']], dict_exp_clean['indices_words']


# Explaining Lime

In [ ]:
sparse_matrix = SparseMatrix()

In [ ]:
c_clean = make_pipeline(vectorizer_clean, sparse_matrix, model_clean)

In [ ]:
from lime.lime_text import LimeTextExplainer
explainer_clean = LimeTextExplainer(class_names=class_names, random_state=65464)

In [ ]:
exp_clean = explainer_clean.explain_instance(newsgroups_test_clean.data[idx]
                                             , c_1.predict_proba
                                             , num_features=10)

In [ ]:
exp_clean.as_list()

In [ ]:
exp_clean.show_in_notebook(text=True)

# Training a RF

In [ ]:
import sklearn.ensemble

In [ ]:
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
rf.fit(train_vectors_clean, newsgroups_train_clean.target)

## Testing RF Model

In [ ]:
pred = rf.predict(test_vectors_clean)
sklearn.metrics.f1_score(newsgroups_test_clean.target, pred, average='binary')

In [ ]:
rf.predict(x_explain_clean)

## Training a NN model

In [ ]:
for parameter in comb_parameters:
    model_1 = NNPredict(
        verbose=1
        , es=True
        , gpu=True
        , dataloader_workers=0
        , **parameter
    )
    model_1.fit(x_train=train_vectors, y_train=newsgroups_train.target)

In [ ]:
pred = model_1.predict(test_vectors)
print('Score:', sklearn.metrics.f1_score(newsgroups_test.target, pred, average='binary'))

In [ ]:
c = make_pipeline(vectorizer, sparse_matrix, model_1) 

In [ ]:
c.predict_proba([newsgroups_test.data[idx]])

In [ ]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names)
class_names

In [ ]:
exp = explainer.explain_instance(newsgroups_test.data[idx], c.predict_proba, num_features=10)

In [ ]:
#exp.as_list()

In [ ]:
exp.show_in_notebook(text=False)

In [ ]:
import numpy as np
from scipy.sparse import issparse

from matplotlib import pyplot as plt


def label_bar(rects, ax, labels=None, offset_y=0.4):
    colors = ['blue', 'orange']
    N = len(rects)
#     for rect, color in zip(rects, colors):
    for i in range(N):
        rect = rects[i]
#         color = colors[i]
        width = rect.get_width()
        text_width = '{:3.2f}'.format(width)
        if labels is None:
            text = text_width
            ax.annotate(text,
                    xy=(rect.get_width() / 2, rect.get_y() - offset_y + rect.get_height() / 2),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points"
                    , ha='center'
                    , va='bottom',
                    size=30)
        else:
            # alterando
            text = labels[i]
            ax.annotate(
                text_width
                , xy=(rect.get_width() / 2, (rect.get_y() + rect.get_height() / 2) - 0.25)
                , xytext=(0, -1)  # 3 points vertical offset
                , textcoords="offset points"
                , ha='center', va='bottom'
                , size=13
                , color='black'
                , horizontalalignment='right'
            )
            if rect.get_width() > 0:
                aling_text = 'right'
                off_setx = -3
            else:
                aling_text = 'left'
                off_setx = +3
           
            ax.annotate(
                text
                , xy=(rect.get_x(), rect.get_y())
                , xytext=(off_setx, 0)  # 3 points vertical offset
                , textcoords="offset points"
                , ha=aling_text
                , va='bottom'
                , size=14)
#         print(rect.get_x(), rect.get_y(), rect.get_height(), rect.get_width())
        
     

    
def simpleaxis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)


class SparseMatrix(object):
    """
    Transformation to be used in a sklearn pipeline
    check if a array is sparse.
    # TODO: The NLS, LLS, NNPredict should accept sparse array
    """
    def __init__(self):
        pass

    def fit(self):
        return self

    @staticmethod
    def transform(x):
        if issparse(x):
            return x.toarray()
        return x


class ExplainText(object):
    def __init__(self, model, class_names, names_features):
        """
        :param model: NLS model;
        :param class_names: class names to be utilized in the plot;
        :param names_features: names of the features.
        """
        self.model = model
        self.class_names = np.array(class_names)
        self.names_features = np.array(names_features)

    def get_text_explanation(self, x_explain, document, num_features=10):
        """
        Get the explanation of text document.
        :param x_explain: document to be explained, should be vectorized;
        :param document: document in text format;
        :param num_features: number of features to produce the explanation.
        :return: betas values and words correspondent to the explanation.
        """
        explanation = self.model.get_thetas(x_pred=x_explain, net_scale=True)
        betas = explanation[2][0]
        words_from_text_indices = np.argwhere(x_explain[0] != 0).reshape(-1)
                
        # Prediction from the model
        prediction = self.model.predict(x_explain).reshape(-1)
        predict_proba = self.model.predict_proba(x_explain).reshape(-1)
        ind_pred_proba = np.argsort(predict_proba)[::-1]

        # Get the col_number of the predict and the second classes.
        col_betas = ind_pred_proba[0]
        col_betas_neg = ind_pred_proba[1]
        
        # Get the betas for the predict and second (neg) classes.
        betas_document = betas[words_from_text_indices, col_betas]
        betas_document_neg = betas[words_from_text_indices, col_betas_neg]

        betas_final = betas_document - betas_document_neg
        words_features_document = self.names_features[words_from_text_indices].reshape(-1)

        # Ranking the betas by absolute value - crescent order.
        beta_0_abs = np.abs(betas_final)
        betas_rank_ind = np.argsort(beta_0_abs)[::-1]
        # Selecting the first num_features important features.
        betas_rank_ind = betas_rank_ind[:num_features] 
        
        # Geting the important words.
        words_features_document_rank = words_features_document[betas_rank_ind]

        return dict(betas=betas_final[betas_rank_ind]
                    , betas_document=betas_document[betas_rank_ind]
                    , betas_document_neg=betas_document_neg[betas_rank_ind]
                    , words=words_features_document_rank
                    , prediction=prediction
                    , prediction_proba=predict_proba
                    , ind_class_sorted=ind_pred_proba
                    , document=document
                    , indices_words=words_from_text_indices[betas_rank_ind]
                    , indices_all_words=words_from_text_indices
                    )

    def document_html(self, x_explain, document, num_features=10, tokenizer=None):
        exp = self.get_text_explanation(x_explain, document, num_features=num_features)
        if tokenizer is None:
            return None
        document_html = ''
        document_tokens = tokenizer(document)
        for words in document_tokens:
            if words in exp['words']:
                document_html += words
            print(words)


    def explain_graphical(self, x_explain, document, num_features=10):
        exp = self.get_text_explanation(x_explain, document, num_features=num_features)
        fig, axs = plt.subplots(1, 3, figsize=(12, 5), dpi=200)
        colors = ['green', 'red']
        rects1 = axs[0].barh(self.class_names[::-1], exp['prediction_proba'][::-1], color=colors[::-1])
        axs[0].set_ylim([-3, 2])
        simpleaxis(axs[0])
        axs[0].set_xticks([])
        axs[0].tick_params('y', labelsize=20)
#         for rect, color in zip(rects1, colors):
#             rect.set_color(color)
        axs[0].set_title('Predicted Probabilities', size=20)
        
        label_bar(rects1, axs[0])
        names = exp['words'][::-1]
        vals = exp['betas'][::-1]
        class_names_sorted = self.class_names[exp['ind_class_sorted']]
        self.get_plot_feature_importance(axs[1], names, vals, class_names_sorted)
        
        
        simpleaxis(axs[2])
        axs[2].set_xticks([])
        axs[2].set_yticks([])
        axs[2].text(0, 1, '\n' + exp['document'], style='italic', wrap=True, va='top')
        axs[2].set_title('Document to Explain', size=20)
        plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.5, hspace=None)
        return fig, axs

    def get_plot_feature_importance(self, ax, names, vals, class_names):
        colors = ['green' if x > 0 else 'red' for x in vals]
        pos = np.arange(len(vals))
        rects2 = ax.barh(pos, vals, align='center', color=colors)
        ax.set_yticks([])
        ax.set_yticklabels([])
        ax.set_xticks([])
        ax.set_xticklabels([])
        label_bar(rects2, ax, labels=names)
        ax.axvline(0, color='black', lw=2)
        ax.set_title('Features Importance', size=20)
        y_lim = np.array(ax.get_ylim())
        ax.set_ylim(y_lim+np.array([0, 1.6]))
        
        ax.annotate('  ' + class_names[0], xy=(0,y_lim[1]), size=22, color='green', ha='left')
        ax.annotate(class_names[1] + '  ', xy=(0,y_lim[1]), size=22, color='red', ha='right')
        simpleaxis(ax)

In [ ]:
exp_NLS = ExplainText(model, class_names=['atheism', 'christian'], names_features=names_features)
# dict_exp = exp.get_text_explanation(x_explain, document=document_explain)

In [ ]:
dict_exp = exp_NLS.get_text_explanation(x_explain, document=document_explain, num_features=10)

In [ ]:
plot_exp = exp_NLS.explain_graphical(x_explain, document=document_explain, num_features=10)
